In [1]:
from paddleocr import PaddleOCR,draw_ocr
import cv2
from PIL import Image, ImageFont
import numpy as np
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
img_path = '../destijl_dataset/04_text/0328.png'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)


result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
image = cv2.imread(img_path)

for bbox in boxes:
    x, y = bbox[0][0], bbox[0][1]
    z, t = bbox[2][0], bbox[2][1]
    cv2.rectangle(image, (int(x), int(y)), (int(z), int(t)), (0, 255, 0), 2)
    
cv2.imwrite('result.jpg', image)

pixels = np.float32(image.reshape(-1, 3))

n_colors = 4
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
palette = np.asarray(palette, dtype=np.int64)
palette_w_white = []

for color in palette:
    x, y, z = color
    if not (252 < x < 256 and 252 < y < 256 and 252 < z < 256):
        palette_w_white.append(color)

_, counts = np.unique(labels, return_counts=True)
dominant = palette_w_white[np.argmax(counts)]


KeyboardInterrupt: 

In [3]:
from paddleocr import PaddleOCR
import cv2
from PIL import Image, ImageFont
import numpy as np

def compose_paragraphs(text_bboxes, text_palettes):
    num_text_boxes = len(text_palettes)
    composed_text_idxs = [[0]]
    for i in range(num_text_boxes-1):
        palette1 = text_palettes[i]
        palette2 = text_palettes[i+1]
        if np.array_equal(palette1, palette2):
            bbox1 = text_bboxes[i]
            bbox2 = text_bboxes[i+1]
            height1 = bbox1[0][1] - bbox1[3][1]
            height2 = bbox2[0][1] - bbox2[3][1]
            if abs(bbox1[0][1]-bbox2[0][1]) <= abs(height1)+30:
                if i != 0 and i not in composed_text_idxs[-1]:
                    composed_text_idxs.append([i])
                composed_text_idxs[-1].append(i+1)
            else:
                if i != 0 and i not in composed_text_idxs[-1]:
                    composed_text_idxs.append([i])
                if i == num_text_boxes-2:
                    composed_text_idxs.append([i+1])
        else:
            if i != 0 and i not in composed_text_idxs[-1]:
                composed_text_idxs.append([i])
            if i == (num_text_boxes-2):
                composed_text_idxs.append([i+1])

    return composed_text_idxs

def merge_bounding_boxes(composed_text_idxs, bboxes):
        '''
            openCV --> x: left-to-right, y: top--to-bottom
            bbox coordinates --> [[256.0, 1105.0], [1027.0, 1105.0], [1027.0, 1142.0], [256.0, 1142.0]]
                             --> left top, right top, right bottom, left bottom
        '''
        
        biggest_borders = []
        for idxs in composed_text_idxs:
            smallest_x = smallest_y = 10000
            biggest_y = biggest_x = 0
            if len(idxs) > 1:
                for idx in idxs:
                    bbox = bboxes[idx]
                    bbox_smallest_x, bbox_smallest_y = np.min(bbox, axis=0)
                    bbox_biggest_x, bbox_biggest_y = np.max(bbox, axis=0)

                    if smallest_x > bbox_smallest_x:
                        smallest_x = bbox_smallest_x
                    if smallest_y > bbox_smallest_y:
                        smallest_y = bbox_smallest_y
                    if biggest_x < bbox_biggest_x:
                        biggest_x = bbox_biggest_x
                    if biggest_y < bbox_biggest_y:
                        biggest_y =  bbox_biggest_y

                biggest_border = [[smallest_x, smallest_y], [biggest_x, smallest_y], [biggest_x, biggest_y], [smallest_x, biggest_y]]
                biggest_borders.append(biggest_border)
            else:
                biggest_borders.append(bboxes[idxs[0]])
        return biggest_borders

# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
img_path = '../destijl_dataset/04_text/0327.png'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for i, line in enumerate(res):
        print(i, line)

result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
image = cv2.imread(img_path)

n_colors = 3
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
flags = cv2.KMEANS_RANDOM_CENTERS

county = 0
palettes = []
for bbox in boxes:
    x, y = int(bbox[0][0]), int(bbox[0][1])
    z, t = int(bbox[2][0]), int(bbox[2][1])
    #cv2.rectangle(image, (x, y), (z, t), (0, 255, 0), 2)
    cropped_image = image[y:t, x:z]
    pixels = np.float32(cropped_image.reshape(-1, 3))

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    palette = np.asarray(palette, dtype=np.int64)
    palette_w_white = []

    for i, color in enumerate(palette):
        x, y, z = color
        if not (252 < x < 256 and 252 < y < 256 and 252 < z < 256):
            palette_w_white.append(color)
        else:
            labels = np.delete(labels, np.where(labels == i))

    _, counts = np.unique(labels, return_counts=True)
    dominant = palette_w_white[np.argmax(counts)]
    palettes.append(palette_w_white)

composed = compose_paragraphs(boxes, palettes)

biggest = merge_bounding_boxes(composed, boxes)
print(composed)
image2 = cv2.imread(img_path)
print(img_path)
for bbox in biggest:
    x, y = int(bbox[0][0]), int(bbox[0][1])
    z, t = int(bbox[2][0]), int(bbox[2][1])
    cv2.rectangle(image2, (x, y), (z, t), (0, 255, 0), 2)
cv2.imwrite('result.jpg', image2)

[2023/07/06 13:30:05] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/busraasan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/busraasan/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.

True

In [ ]:
counts